In [1]:
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.tensorflow import TensorFlow
import boto3
import random
import os
import config
import json
import pandas as pd
import datetime
import gzip
import io

In [ ]:
S3_BUCKET = 'wsbidder'
CSV_SEPARATOR = '\t'
SRC_BUCKET = 'wsbidder'
DST_BUCKET = 'wsbidder'

SRC_PREFIX = 'tsv/etl/imp-pred-service-v1/imppredservice_training_data'
DST_PREFIX = 'trainer_predict_bid/data'

In [12]:
%%time

s3_resource = boto3.resource('s3')
new_train_files = [ obj.key for obj in s3_resource.Bucket(DST_BUCKET).objects.filter(Prefix='{}/train/'.format(DST_PREFIX)).all() if obj.key.endswith('.gz') ]
# print(new_train_files)
file_paths = [ 's3://{}/{}/train/{}'.format(DST_BUCKET, DST_PREFIX, os.path.basename(filename)) for filename in new_train_files ]
# print(file_paths)
CONFIG = config.get_config()
dtypes = config.get_types_of_attributes()
df = pd.concat((pd.read_csv(f, sep=CSV_SEPARATOR, compression='gzip', na_values=["null", "\\N"], dtype=dtypes) for f in file_paths))
df['targetbid'] = df['targetbid'].astype(float)
won_bids = df[df['won'] == 1]
lost_bids = df[df['won'] == 0]

CPU times: user 9.42 s, sys: 309 ms, total: 9.73 s
Wall time: 9.91 s
